In [ ]:
#!pip install ipynb

In [ ]:
from ipynb.fs.full.fail_stop_agreement import agreement, Process
import threading

n = 4

In [ ]:
def test_same_input(input):
    threads = []
    processes = []

    for i in range(0, n):
        pr = Process(i, input)
        processes.append(pr)

        thr = threading.Thread(target=agreement, args=((pr,)))
        threads.append(thr)

    for thr in threads:    
        thr.start()

    for thr in threads:
        thr.join()

    print("*******   SOLUTION:   *******")
    for pr in processes:
        print("process(", pr.id, ") = ", pr.output, " @epoch: ", pr.decision_epoch)
        if pr.output != input or pr.decision_epoch != 1:
            return False
    print()
    return True

In [ ]:
# During each epoch, both of the values 0 and 1 are never sent in any execution of round 2
def test_lemma_9():
    threads = []
    processes = []

    for i in range(0, n):
        pr = Process(i, str(i%2))
        processes.append(pr)

        thr = threading.Thread(target=agreement, args=((pr,)))
        threads.append(thr)

    for thr in threads:    
        thr.start()

    for thr in threads:
        thr.join()

    # Pre-processing: get all round 2 messages per epoch
    epochs_num = [pr.decision_epoch+1 for pr in processes]
    total_epochs = max(epochs_num)
    
    round2_msgs_per_epoch = [[]] * total_epochs

    for pr in processes:
        for msg in pr.broadcasted_messages:
            if msg.round == 2:
                round2_msgs_per_epoch[msg.epoch-1].append(msg.message)      # epoch numbers start at 1 but list positions start at 0
    
    for epoch_msgs in round2_msgs_per_epoch:
        ocurrences_0 = epoch_msgs.count("0")
        ocurrences_1 = epoch_msgs.count("1")
        if ocurrences_0 > 0 and ocurrences_1 > 0:
            return False
    return True



In [ ]:
#print(test_same_input("0"))
#print(test_same_input("1"))
print(test_lemma_9())